## Face Recognition

Zumi can be trained to recognize your face! In this demo, we will give Zumi pictures of your face, have her train a model, and then run the model.

### Step 1: Setup

In Jupyter Notebook, code is written inside boxes called "cells". You can run a cell by clicking on the small black arrow next to it. Try running the cell below, which will set Zumi up to start recognizing faces.

In [ ]:
import sys
sys.path.insert(0,'/home/pi/Zumi_Contents/Datas/Face_recognition')
from recognition import Recognition

from zumi.util.camera import Camera
from zumi.util.screen import Screen

import time
import cv2
import IPython.display
import PIL.Image
import numpy as np
import os

fd = Recognition()
camera = Camera(auto_start=False)
screen = Screen()

Did you notice the "In \[ \]:" text next to the cell? While the code in a cell is still running, there will be an asterisk inside the box, like this: "In \[\*\]:". You can tell a cell is finished because there will be a number in there instead, like this: "In \[1\]". The number is the number of cells you've ran since you started the notebook, so the first cell you run will have a "1" inside the box, the second cell will have a "2", and so on.

### Step 2: Collect Pictures

This cell will turn on Zumi's camera so that she can start taking pictures of faces.

* Place Zumi somewhere so that your face is level with her camera.
* Enter your name.
* Look into Zumi's camera, but be sure to leave enough room so she can see your face!
* Once she sees your face, she will take a picture and remember it for later. You can tell she can see your face because a box will be drawn around it on her screen.
* Keep looking into Zumi's camera until her screen says "Done!".

In [ ]:
def collectPictures():
    camera.start_camera()
    fd.name = input("input name : ")
    print("Capture 50 pictures")
    time.sleep(1)
    
    while True:
        try:
            image = camera.capture()
            fd.makeDataset(image)
            IPython.display.display(PIL.Image.fromarray(image))
            screen.draw_image(PIL.Image.fromarray(fd.streaming_image).convert('1'))
            IPython.display.clear_output(wait=True)
            
            if fd.cap > 50:
                print("done")
                screen.draw_text_center("Done!")
                camera.close()
                break
        except Exception as e:
            print(e)
            screen.draw_text("")
            camera.close()
            break
            
    fd.cap = 0
    time.sleep(1)
    screen.draw_text("")
    camera.close()
    
collectPictures()

Zumi can learn more than one face, by the way. Try running the cell again with a different person.

### Step 3: Train a Model

Once Zumi has enough pictures of faces, she will need time to learn the different faces and sort them out.

In computer language, this process is called "training". The purpose of this training is to create a "model", which Zumi will be using in the next step to recognize the faces that she learned.

In [ ]:
print("Training model...")

fd.trainModel()

print("Done!")

### Step 4: Run Model

How did Zumi do? Run this cell, and Zumi will start to recognize faces using the trained model from the previous step.

Once this cell is ran, it will stop running automatically after 30 seconds. You can end it earlier by clicking on the cell, and then clicking the black square in the toolbar.

In [ ]:
def runModel():
    camera.start_camera()
    print("loading...")
    fd.recognizer.read('../Datas/Face_recognition/trainer/trainer.yml')
    print("start")
    
    currentTime = time.time()
    
    while time.time() - currentTime < 30:
        try:
            image = camera.capture()
            fd.recognition(image)
            IPython.display.display(PIL.Image.fromarray(image))
            screen.draw_image(PIL.Image.fromarray(fd.streaming_image).convert('1'))
            IPython.display.clear_output(wait=True)
        except Exception as e:
            print(e)
            screen.draw_text("")
            camera.close()
            break
    camera.close()
    
runModel()

### Step 5: Cleanup

All done? Shutdown the camera and screen so they can be used for another demo.

In [ ]:
camera.close()
screen.draw_text("")

### Options

These cells aren't necessary for the demo, but they may come in hand.

#### Delete One Dataset

Not satisfied with your pictures? Run this cell to remove one set of faces from Zumi's memory.

In [ ]:
import pickle

labels = 0    
try:
    with open('../Datas/Face_recognition/labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)
    names = list(labels.values())
    print(names)
    name = input("Please enter a name: ")
    print("?")
    fd.deleteDataset(name)
    print("??")
except:
    print("[Error] No dataset")

#### Delete All Datasets

Want to reset the demo? Run this cell to wipe Zumi's memory clean of all traces of faces.

In [ ]:
fd.deleteAllDatasets()